In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


In [1]:
pip install minisom


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
# t-SNE visualization
from sklearn.manifold import TSNE

# imputation
from sklearn.impute import KNNImputer

# Scaling
from sklearn.preprocessing import StandardScaler

# PCA
from sklearn.decomposition import PCA

# K-means for Clustering
from sklearn.cluster import KMeans

# cluster metrics
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score

# Calculate silhouette_score
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")


In [3]:
#Read files
trainfile = r'CC GENERAL.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe


print(trainData.shape)
trainData.head()


(8950, 18)


,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [4]:
trainData.isna().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [5]:
def outlier_percent(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    minimum = Q1 - (1.5 * IQR)
    maximum = Q3 + (1.5 * IQR)
    num_outliers =  np.sum((data < minimum) |(data > maximum))
    num_total = data.count()
    return (num_outliers/num_total)*100

In [6]:
non_categorical_data = trainData.drop(['CUST_ID'], axis=1)
for column in non_categorical_data.columns:
    data = non_categorical_data[column]
    percent = str(round(outlier_percent(data), 2))
    print(f'Outliers in "{column}": {percent}%')

Outliers in "BALANCE": 7.77%
Outliers in "BALANCE_FREQUENCY": 16.68%
Outliers in "PURCHASES": 9.03%
Outliers in "ONEOFF_PURCHASES": 11.32%
Outliers in "INSTALLMENTS_PURCHASES": 9.69%
Outliers in "CASH_ADVANCE": 11.51%
Outliers in "PURCHASES_FREQUENCY": 0.0%
Outliers in "ONEOFF_PURCHASES_FREQUENCY": 8.74%
Outliers in "PURCHASES_INSTALLMENTS_FREQUENCY": 0.0%
Outliers in "CASH_ADVANCE_FREQUENCY": 5.87%
Outliers in "CASH_ADVANCE_TRX": 8.98%
Outliers in "PURCHASES_TRX": 8.56%
Outliers in "CREDIT_LIMIT": 2.77%
Outliers in "PAYMENTS": 9.03%
Outliers in "MINIMUM_PAYMENTS": 9.74%
Outliers in "PRC_FULL_PAYMENT": 16.47%
Outliers in "TENURE": 15.26%


In [7]:
for column in non_categorical_data.columns:
    data = non_categorical_data[column]
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    minimum = Q1 - (1.5 * IQR)
    maximum = Q3 + (1.5 * IQR)
 
    outliers = ((data < minimum) |(data > maximum))
    non_categorical_data[column].loc[outliers] = np.nan
    
non_categorical_data.isna().sum()

BALANCE                              695
BALANCE_FREQUENCY                   1493
PURCHASES                            808
ONEOFF_PURCHASES                    1013
INSTALLMENTS_PURCHASES               867
CASH_ADVANCE                        1030
PURCHASES_FREQUENCY                    0
ONEOFF_PURCHASES_FREQUENCY           782
PURCHASES_INSTALLMENTS_FREQUENCY       0
CASH_ADVANCE_FREQUENCY               525
CASH_ADVANCE_TRX                     804
PURCHASES_TRX                        766
CREDIT_LIMIT                         249
PAYMENTS                             808
MINIMUM_PAYMENTS                    1154
PRC_FULL_PAYMENT                    1474
TENURE                              1366
dtype: int64

In [8]:
imputer = KNNImputer()
imp_data = pd.DataFrame(imputer.fit_transform(non_categorical_data), columns=non_categorical_data.columns)
imp_data.isna().sum()

BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

In [12]:
std_imp_data = pd.DataFrame(StandardScaler().fit_transform(imp_data), columns=imp_data.columns)
std_imp_data.head(5)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.938415,-2.648163,-0.769054,-0.668008,-0.453071,-0.648957,-0.806490,-0.744535,-0.707313,-0.747150,-0.681634,-0.780853,-1.058772,-0.990200,-0.849651,-0.449541,0.0
1,1.544889,-1.140713,-0.910890,-0.668008,-0.787709,0.880535,-1.221758,-0.744535,-0.916995,1.029213,0.865275,-0.966433,0.877124,2.904565,1.216882,2.720181,0.0
2,0.989314,0.366737,0.238620,1.571492,-0.787709,-0.648957,1.269843,2.115073,-0.916995,-0.747150,-0.681634,0.147050,1.038449,-0.570632,0.230934,-0.449541,0.0
3,0.338573,0.065247,1.317747,0.229675,-0.787709,-0.354281,-1.014125,-0.377920,-0.916995,-0.155031,-0.294907,-0.873643,1.038449,-1.191668,1.304805,-0.449541,0.0
4,-0.328254,0.366737,-0.887102,-0.621664,-0.787709,-0.648957,-1.014125,-0.377920,-0.916995,-0.747150,-0.681634,-0.873643,-0.994242,-0.514457,-0.616417,-0.449541,0.0


In [ ]:
#K-Means Clustering ========================================================================
#Default Mode - K=8
trainData1 = pd.DataFrame(std_imp_data['PURCHASES_FREQUENCY'])
kmeans = KMeans()
kmeans.fit(trainData1)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
#Print Silhouette measure
print(silhouette_score(trainData1, kmeans.labels_))
print(trainData1.shape)


In [ ]:
#Add Cluster Number to each datapoint + save file
df_kmeans=trainData1.copy()
kmeans_predict_train=kmeans.predict(trainData1)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(std_imp_data.shape)
print(df_kmeans.shape)
print(df_kmeans.head(20))
df_kmeans.dtypes

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-8.csv')


In [ ]:
#K-Means Clustering ========================================================================
# K=20

kmeans1 = KMeans(n_clusters=20, init='random', algorithm='elkan')
kmeans1.fit(trainData1)
kmeans1_predict_train=kmeans1.predict(trainData1)
print(kmeans1_predict_train)

#print(kmeans.labels_)
print(kmeans1.cluster_centers_)
print(trainData1.shape)
#Print Silhouette measure
print(silhouette_score(trainData1, kmeans1.labels_))

In [ ]:
#Add Cluster Number to each datapoint + save file
df_kmeans=trainData1.copy()
kmeans1_predict_train=kmeans.predict(trainData1)
print(kmeans1_predict_train)

df_kmeans['Cluster Number']=kmeans1_predict_train
print(df_kmeans.shape)
print(df_kmeans.head())
print(df_kmeans.dtypes)

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-20.csv')


In [ ]:
#Add Cluster Number to each datapoint + save file

kmeans2 = KMeans(n_clusters=15, init='random', algorithm='elkan')
kmeans2.fit(trainData1)
kmeans2_predict_train=kmeans2.predict(trainData1)
print(kmeans2_predict_train)
#print(kmeans.labels_)
print(kmeans2.cluster_centers_)
print(trainData1.shape)
#Print Silhouette measure
print(silhouette_score(trainData1, kmeans2.labels_))



In [ ]:
#Add Cluster Number to each datapoint + save file
df_kmeans=trainData1.copy()
kmeans2_predict_train=kmeans2.predict(trainData1)
print(kmeans2_predict_train)

df_kmeans['Cluster Number']=kmeans2_predict_train
print(df_kmeans.shape)
print(df_kmeans.head())
print(df_kmeans.dtypes)

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-15.csv')

In [10]:
#K-Means Clustering ========================================================================
#Default Mode - K=8
trainData2 = pd.DataFrame(std_imp_data['BALANCE'])
kmeans = KMeans()
kmeans.fit(trainData2)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
#Print Silhouette measure
print(silhouette_score(trainData2, kmeans.labels_))
print(trainData2.shape)


[0 1 6 ... 0 0 7]
[[-0.90669118]
 [ 1.81454533]
 [-0.17493044]
 [ 0.71053388]
 [ 0.23498302]
 [ 2.47880813]
 [ 1.24290572]
 [-0.58845547]]
0.6092279140407517
(8950, 1)


In [11]:
#Add Cluster Number to each datapoint + save file
df_kmeans=trainData2.copy()
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(trainData2.shape)
print(df_kmeans.shape)
print(df_kmeans.head(20))
df_kmeans.dtypes

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General_bal-8.csv')

[0 1 6 ... 0 0 7]
(8950, 1)
(8950, 2)
     BALANCE  Cluster Number
0  -0.938415               0
1   1.544889               1
2   0.989314               6
3   0.338573               4
4  -0.328254               2
5   0.451019               4
6  -0.477849               7
7   0.461877               4
8  -0.173350               2
9  -0.850972               0
10  0.045165               4
11 -0.475073               7
12  0.220955               4
13 -0.246582               2
14  1.207379               6
15  1.220648               6
16  0.657004               3
17 -0.938266               0
18  0.591808               3
19  1.839835               1


In [ ]:
#Add Cluster Number to each datapoint + save file
df_kmeans=trainData2.copy()
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans.labels_
print(trainData2.shape)
print(df_kmeans.shape)
print(df_kmeans.head(20))
df_kmeans.dtypes

#export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-8.csv')


In [14]:
#K-Means Clustering ========================================================================
# K=20

kmeans = KMeans(n_clusters=20, init='random', algorithm='elkan')
kmeans.fit(trainData2)
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

#print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(trainData2.shape)
#Print Silhouette measure
print(silhouette_score(trainData2, kmeans.labels_))

#Add Cluster Number to each datapoint + save file
df_kmeans=trainData2.copy()
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans_predict_train
print(df_kmeans.shape)
print(df_kmeans.head())
print(df_kmeans.dtypes)

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-bal-20.csv')

[16  1  2 ... 16 16  5]
[[-0.16532992]
 [ 1.51861704]
 [ 0.97782311]
 [ 1.84330043]
 [ 0.38876669]
 [-0.6957291 ]
 [-0.04397916]
 [ 0.09773544]
 [ 1.22398331]
 [ 2.63519167]
 [ 2.23196383]
 [-0.29800975]
 [-0.44278919]
 [-0.88582419]
 [ 0.55863894]
 [-0.58137331]
 [-0.95100275]
 [-0.80006363]
 [ 0.7606026 ]
 [ 0.23411178]]
(8950, 1)
0.5531986665735645
[16  1  2 ... 16 16  5]
(8950, 2)
    BALANCE  Cluster Number
0 -0.938415              16
1  1.544889               1
2  0.989314               2
3  0.338573               4
4 -0.328254              11
BALANCE           float64
Cluster Number      int32
dtype: object


In [15]:

kmeans = KMeans(n_clusters=15, init='random', algorithm='elkan')
kmeans.fit(trainData2)
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

#print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(trainData2.shape)
#Print Silhouette measure
print(silhouette_score(trainData2, kmeans.labels_))

#Add Cluster Number to each datapoint + save file
df_kmeans=trainData2.copy()
kmeans_predict_train=kmeans.predict(trainData2)
print(kmeans_predict_train)

df_kmeans['Cluster Number']=kmeans_predict_train
print(df_kmeans.shape)
print(df_kmeans.head())
print(df_kmeans.dtypes)

export_csv = df_kmeans.to_csv(r'KmeansClusters_CC_General-bal-15.csv')

[11  0 13 ... 11 11  4]
[[ 1.6756344 ]
 [ 0.39755838]
 [-0.83129611]
 [ 2.6036056 ]
 [-0.68507524]
 [-0.01318411]
 [ 0.18209303]
 [ 2.14137657]
 [-0.17239193]
 [ 0.63210933]
 [ 1.2700734 ]
 [-0.94167005]
 [-0.52311376]
 [ 0.93335785]
 [-0.33739433]]
(8950, 1)
0.5673854868111196
[11  0 13 ... 11 11  4]
(8950, 2)
    BALANCE  Cluster Number
0 -0.938415              11
1  1.544889               0
2  0.989314              13
3  0.338573               1
4 -0.328254              14
BALANCE           float64
Cluster Number      int32
dtype: object


https://heartbeat.comet.ml/introduction-to-self-organizing-maps-soms-98e88b568f5d

We initialize a 6-by-6 SOM with a learning rate of 0.5.

We train the SOM on 100 iterations.

The next parameter is sigma. Sigma is the radius of the different neighbors in the SOM. The default value for this is 1.0.

MiniSom implements two types of training: train_random and train_batch. In train_random, we train the model by picking random samples from our data. In train_batch, the samples are picked in the order that they are stored. A method called random_weights_init initializes the weights by picking random samples from the data.



